# United outdoors datawarehouse

## Imports

In [79]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Integer, String, LargeBinary, VARCHAR, NVARCHAR, DECIMAL, CHAR, \
    DATE
from sqlalchemy.dialects.mssql import BIT, XML, MONEY, TIME
from sqlalchemy.exc import OperationalError
from urllib import parse
import re
import time

## Starting timer

In [80]:
start_time = time.time()

## Database connection details

In [81]:
DB = {
    'servername' : '(local)\\SQLEXPRESS',
    'united_outdoors_database' : 'UnitedOutdoors',
    'northwind_database' : 'Northwind',
    'aenc_database' : 'Aenc',
    'adventureworks_database' : 'AdventureWorks2019',
    'master' : 'master'
}

In [82]:
def create_connection(servername, database):
    params = parse.quote_plus(f'DRIVER={{SQL Server}};SERVER={servername};DATABASE={database};Trusted_Connection=yes')
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}', use_setinputsizes=False, connect_args={'options': '-c search_path=dbo'}, fast_executemany=True) # setinputsizes needs to be turned off for sql server, idk why but gives errors otherwise
    try:
        establish_conn = engine.connect()
        print(f'Connection to {database} database successful')
        return establish_conn, engine
    except OperationalError as e:
        print(f'Error: {e}')
        return None, None

In [83]:
def split_and_execute_sql_script(script, execute_engine):
    # creating a connection
    connection = execute_engine.connect()
    
    # splitting the script into the database creation and the rest
    commands = re.split(r'GO\n', script)
    # removing all \bGO\b from the commands
    commands = [re.sub(r'\bGO\b', '', command) for command in commands]
    
    # Execute the commands
    for command in commands:
        command = command.strip()
        # Skip if the command is empty or 'GO'
        if not command or command.upper() == 'GO':
            continue
        try:
            connection.connection.execute(command)
            connection.connection.commit()
            #print(f'Command executed: {command}')
        except OperationalError as e:
            print(f'Error: {e} at command: {command}')

In [84]:
def bulk_insert(df, dtypes, table_name, engine):
    # calculating chunk size
    chunk_size = (2000 // len(df.columns))  # 2100 is the maximum number of parameters in a query, -100 for safety

    print(f'Inserting data into table: {table_name} with chunk size: {chunk_size}')
    df.to_sql(name=table_name, schema='dbo', con=engine, if_exists='append', index=False, dtype=dtypes, method='multi', chunksize= chunk_size)

In [85]:
def prepare(dataframe, nk_sk_dict=None):
    # replacing the natural keys with the surrogate keys
    if nk_sk_dict:
        for column in nk_sk_dict:
            # Check for duplicate keys
            if len(nk_sk_dict[column]) != len(set(nk_sk_dict[column])):
                raise ValueError(f'Duplicate keys found in nk_sk_dict for column: {column}')
            else:
                print(f'Replacing natural keys with surrogate keys for column: {column}')
                for natural_key in nk_sk_dict[column]:
                    dataframe[column] = dataframe[column].replace(natural_key, nk_sk_dict[column][natural_key])
    
    # replace empty values with None
    dataframe = dataframe.where(pd.notnull(dataframe), None)
    dataframe = dataframe.replace({np.nan: None})
    
    # stripping all columns with string data
    dataframe = dataframe.map(lambda x: x.strip() if isinstance(x, str) else x)
    # replacing all empty strings with None
    dataframe = dataframe.replace(r'^\s*$', None, regex=True)

In [86]:
def prepare_and_insert(dataframe, dtypes, table_name, insert_engine, nk_sk_dict=None):
    """
    Prepares the dataframe for insertion into the database and inserts it into the database.
        @param dataframe: The dataframe to be inserted into the database
        @param dtypes: The data types of the columns in the dataframe
        @param table_name: The name of the table in the database
        @param insert_engine: The engine to insert the data into
        @param nk_sk_dict: A 3d dictionary containing the natural keys and their corresponding surrogate keys, per column (so nk_sk_dict[column][natural_key] = surrogate_key)
    """
       
    prepare(dataframe, nk_sk_dict)
    
    # adding the data to the database
    bulk_insert(dataframe, dtypes, table_name, insert_engine)

In [87]:
def prepare_and_insert_return_sk(dataframe, dtypes, table_name, insert_engine, natural_key_column, nk_sk_dict=None):
    """
    Prepares the dataframe for insertion into the database and inserts it into the database.
        @param dataframe: The dataframe to be inserted into the database
        @param dtypes: The data types of the columns in the dataframe
        @param table_name: The name of the table in the database
        @param insert_engine: The engine to insert the data into
        @param natural_key_column: The name of the column containing the natural keys
        @param nk_sk_dict: A 3d dictionary containing the natural keys and their corresponding surrogate keys, per column (so nk_sk_dict[column][natural_key] = surrogate_key)
        @return: A dictionary containing the natural keys and their corresponding surrogate keys
    """
    
    prepare(dataframe, nk_sk_dict)
    
    # adding the data to the database
    bulk_insert(dataframe, dtypes, table_name, insert_engine)
    
    # making a cleaned dictionary without None/nan values in the natural_key_column
    mask = dataframe[natural_key_column].notnull()
    filtered_dataframe = dataframe[mask]
    
    # getting the natural keys and their corresponding surrogate keys
    # TODO FIX THIS, relying on the dataframe index is not a good idea
    nk_sk_dict = dict(zip(filtered_dataframe[natural_key_column], dataframe.index))
    
    # adding +1 to the index to get the surrogate key
    nk_sk_dict = {k: v + 1 for k, v in nk_sk_dict.items()}
    
    return nk_sk_dict

In [88]:
def prepare_and_update(table_name, update_engine, nk_sk_dict=None):   
    # creating a connection
    connection = update_engine.connect()

    # Start a transaction
    trans = connection.begin()

    try:
        # adding the data to the database
        print(f'Updating data in table: {table_name}')
        for column in nk_sk_dict:
            # Prepare a batch update statement
            update_stmts = []
            for natural_key in nk_sk_dict[column]:
                # TODO this would not work with updated data in datawarehouse, since this would replace the old and new data. Maybe check the datetime or something
                statement = f'UPDATE {table_name} SET {column} = {nk_sk_dict[column][natural_key]} WHERE {column} = {int(natural_key)}'
                update_stmts.append(statement)

                # If the number of parameters reaches the limit, execute the batch update and clear the list
                if len(update_stmts) * 2 >= 2000:  # Each update statement has 2 parameters, cause of the Column and Where clause
                    connection.connection.execute(";".join(update_stmts))
                    update_stmts = []

            # Execute the remaining update statements
            if update_stmts:
                connection.connection.execute(";".join(update_stmts))

        # Commit the transaction
        trans.commit()
    except:
        # Rollback the transaction in case of error
        trans.rollback()
        raise
    finally:
        # Close the connection
        connection.close()

In [89]:
def drop_modified_date_rowguid(dataframe):
    # dropping all columns with 'rowguid' in their name
    columns_to_drop_mr = dataframe.filter(like='rowguid').columns
    
    # dropping all columns with 'ModifiedDate' in their name
    columns_to_drop_mr = columns_to_drop_mr.append(dataframe.filter(like='ModifiedDate').columns)
    
    # dropping the columns
    dataframe.drop(columns=columns_to_drop_mr, inplace=True)

## Create the UnitedOutdoors datawarehouse

In [90]:
_, creation_engine = create_connection(DB["servername"], DB["master"])

# Open the SQL script file and read its contents
with open('sql/UnitedOutdoors_creation.sql', 'r') as file:
    sql_script = file.read()

split_and_execute_sql_script(sql_script, creation_engine)

creation_engine.dispose()

Connection to master database successful


## Connecting to the UnitedOutdoors datawarehouse

In [91]:
_ , united_outdoors_engine = create_connection(DB["servername"], DB["united_outdoors_database"])

Connection to UnitedOutdoors database successful


## Loading the data from the source databases

### Northwind database

#### Connection

In [92]:
northwind_conn, northwind_engine = create_connection(DB["servername"], DB["northwind_database"])

Connection to Northwind database successful


#### Loading data

In [93]:
# Load the data from the source database
northwind_categories = pd.read_sql('SELECT * FROM Categories', northwind_conn)
northwind_customer_customer_demo = pd.read_sql('SELECT * FROM CustomerCustomerDemo', northwind_conn)
northwind_customer_demographics = pd.read_sql('SELECT * FROM CustomerDemographics', northwind_conn)
northwind_customers = pd.read_sql('SELECT * FROM Customers', northwind_conn)
northwind_employees = pd.read_sql('SELECT * FROM Employees', northwind_conn)
northwind_employee_territories = pd.read_sql('SELECT * FROM EmployeeTerritories', northwind_conn)
northwind_order_details = pd.read_sql('SELECT * FROM [Order Details]', northwind_conn)
northwind_orders = pd.read_sql('SELECT * FROM Orders', northwind_conn)
northwind_products = pd.read_sql('SELECT * FROM Products', northwind_conn)
northwind_region = pd.read_sql('SELECT * FROM Region', northwind_conn)
northwind_shippers = pd.read_sql('SELECT * FROM Shippers', northwind_conn)
northwind_suppliers = pd.read_sql('SELECT * FROM Suppliers', northwind_conn)
northwind_territories = pd.read_sql('SELECT * FROM Territories', northwind_conn)

northwind_conn.close()

### Aenc database

#### Connection

In [94]:
aenc_conn , aenc_engine = create_connection(DB["servername"], DB["aenc_database"])

Connection to Aenc database successful


#### Loading data

In [95]:
aenc_bonus = pd.read_sql('SELECT * FROM Bonus', aenc_conn)
aenc_customer = pd.read_sql('SELECT * FROM customer', aenc_conn)
aenc_department = pd.read_sql('SELECT * FROM department', aenc_conn)
aenc_employee = pd.read_sql('SELECT * FROM Employee', aenc_conn)
aenc_product = pd.read_sql('SELECT * FROM product', aenc_conn)
aenc_region = pd.read_sql('SELECT * FROM region', aenc_conn)
aenc_sales_order = pd.read_sql('SELECT * FROM sales_order', aenc_conn)
aenc_sales_order_item = pd.read_sql('SELECT * FROM sales_order_item', aenc_conn)
aenc_state = pd.read_sql('SELECT * FROM state', aenc_conn)

aenc_conn.close()

### AdventureWorks database

#### Connection

In [96]:
adventureworks_conn, adventureworks_engine = create_connection(DB["servername"], DB["adventureworks_database"])

Connection to AdventureWorks2019 database successful


#### Loading data

In [97]:
adventureworks_humanresources_department = pd.read_sql('SELECT * FROM HumanResources.Department', adventureworks_conn)
adventureworks_humanresources_employee = pd.read_sql('SELECT * FROM HumanResources.Employee', adventureworks_conn)
adventureworks_humanresources_employeedepartmenthistory = pd.read_sql('SELECT * FROM HumanResources.EmployeeDepartmentHistory', adventureworks_conn)
adventureworks_humanresources_employeepayhistory = pd.read_sql('SELECT * FROM HumanResources.EmployeePayHistory', adventureworks_conn)
adventureworks_humanresources_jobcandidate = pd.read_sql('SELECT * FROM HumanResources.JobCandidate', adventureworks_conn)
adventureworks_humanresources_shift = pd.read_sql('SELECT * FROM HumanResources.Shift', adventureworks_conn)

In [98]:
adventureworks_person_address = pd.read_sql('SELECT AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, CAST(SpatialLocation AS VARCHAR(MAX)) AS SpatialLocation,rowguid, ModifiedDate   FROM Person.Address', adventureworks_conn)
adventureworks_person_address_type = pd.read_sql('SELECT * FROM Person.AddressType', adventureworks_conn)
adventureworks_person_businessentity = pd.read_sql('SELECT * FROM Person.BusinessEntity', adventureworks_conn)
adventureworks_person_businessentityaddress = pd.read_sql('SELECT * FROM Person.BusinessEntityAddress', adventureworks_conn)
adventureworks_person_businessentitycontact = pd.read_sql('SELECT * FROM Person.BusinessEntityContact', adventureworks_conn)
adventureworks_person_contacttype = pd.read_sql('SELECT * FROM Person.ContactType', adventureworks_conn)
adventureworks_person_countryregion = pd.read_sql('SELECT * FROM Person.CountryRegion', adventureworks_conn)
adventureworks_person_emailaddress = pd.read_sql('SELECT * FROM Person.EmailAddress', adventureworks_conn)
adventureworks_person_password = pd.read_sql('SELECT * FROM Person.Password', adventureworks_conn)
adventureworks_person_person = pd.read_sql('SELECT * FROM Person.Person', adventureworks_conn)
adventureworks_person_personphone = pd.read_sql('SELECT * FROM Person.PersonPhone', adventureworks_conn)
adventureworks_person_phonenumbertype = pd.read_sql('SELECT * FROM Person.PhoneNumberType', adventureworks_conn)
adventureworks_person_stateprovince = pd.read_sql('SELECT * FROM Person.StateProvince', adventureworks_conn)

In [99]:
adventureworks_production_bill_of_materials = pd.read_sql('SELECT * FROM Production.BillOfMaterials', adventureworks_conn)
adventureworks_production_culture = pd.read_sql('SELECT * FROM Production.Culture', adventureworks_conn)
adventureworks_production_document = pd.read_sql('SELECT * FROM Production.Document', adventureworks_conn)
adventureworks_production_illustration = pd.read_sql('SELECT * FROM Production.Illustration', adventureworks_conn)
adventureworks_production_location = pd.read_sql('SELECT * FROM Production.Location', adventureworks_conn)
adventureworks_production_product = pd.read_sql('SELECT * FROM Production.Product', adventureworks_conn)
adventureworks_production_productcategory = pd.read_sql('SELECT * FROM Production.ProductCategory', adventureworks_conn)
adventureworks_production_productcosthistory = pd.read_sql('SELECT * FROM Production.ProductCostHistory', adventureworks_conn)
adventureworks_production_productdescription = pd.read_sql('SELECT * FROM Production.ProductDescription', adventureworks_conn)
adventureworks_production_productdocument = pd.read_sql('SELECT * , CAST(DocumentNode AS VARCHAR(MAX)) AS DocumentNodeString  FROM Production.ProductDocument', adventureworks_conn)
adventureworks_production_productinventory = pd.read_sql('SELECT * FROM Production.ProductInventory', adventureworks_conn)
adventureworks_production_productlistpricehistory = pd.read_sql('SELECT * FROM Production.ProductListPriceHistory', adventureworks_conn)
adventureworks_production_productmodel = pd.read_sql('SELECT * FROM Production.ProductModel', adventureworks_conn)
adventureworks_production_productmodelillustration = pd.read_sql('SELECT * FROM Production.ProductModelIllustration', adventureworks_conn)
adventureworks_production_productmodelproductdescriptionculture = pd.read_sql('SELECT * FROM Production.ProductModelProductDescriptionCulture', adventureworks_conn)
adventureworks_production_productphoto = pd.read_sql('SELECT ProductPhotoID, CONVERT(VARCHAR(MAX),ThumbNailPhoto, 1) as ThumbNailPhotoHexString, ThumbNailPhotoFileName, CONVERT(VARCHAR(MAX), LargePhoto, 1) as LargePhotoHexString, LargePhotoFileName, ModifiedDate FROM Production.ProductPhoto', adventureworks_conn)
adventureworks_production_productproductphoto = pd.read_sql('SELECT * FROM Production.ProductProductPhoto', adventureworks_conn)
adventureworks_production_productreview = pd.read_sql('SELECT * FROM Production.ProductReview', adventureworks_conn)
adventureworks_production_productsubcategory = pd.read_sql('SELECT * FROM Production.ProductSubcategory', adventureworks_conn)
adventureworks_production_scrapreason = pd.read_sql('SELECT * FROM Production.ScrapReason', adventureworks_conn)
adventureworks_production_transactionhistory = pd.read_sql('SELECT * FROM Production.TransactionHistory', adventureworks_conn)
adventureworks_production_transactionhistoryarchive = pd.read_sql('SELECT * FROM Production.TransactionHistoryArchive', adventureworks_conn)
adventureworks_production_unitmeasure = pd.read_sql('SELECT * FROM Production.UnitMeasure', adventureworks_conn)
adventureworks_production_workorder = pd.read_sql('SELECT * FROM Production.WorkOrder', adventureworks_conn)
adventureworks_production_workorderrouting = pd.read_sql('SELECT * FROM Production.WorkOrderRouting', adventureworks_conn)

In [100]:
adventureworks_purchasing_productvendor = pd.read_sql('SELECT * FROM Purchasing.ProductVendor', adventureworks_conn)
adventureworks_purchasing_purchaseorderdetail = pd.read_sql('SELECT * FROM Purchasing.PurchaseOrderDetail', adventureworks_conn)
adventureworks_purchasing_purchaseorderheader = pd.read_sql('SELECT * FROM Purchasing.PurchaseOrderHeader', adventureworks_conn)
adventureworks_purchasing_shipmethod = pd.read_sql('SELECT * FROM Purchasing.ShipMethod', adventureworks_conn)
adventureworks_purchasing_vendor = pd.read_sql('SELECT * FROM Purchasing.Vendor', adventureworks_conn)

In [101]:
adventureworks_sales_countryregioncurrency = pd.read_sql('SELECT * FROM Sales.CountryRegionCurrency', adventureworks_conn)
adventureworks_sales_creditcard = pd.read_sql('SELECT * FROM Sales.CreditCard', adventureworks_conn)
adventureworks_sales_currency = pd.read_sql('SELECT * FROM Sales.Currency', adventureworks_conn)
adventureworks_sales_currencyrate = pd.read_sql('SELECT * FROM Sales.CurrencyRate', adventureworks_conn)
adventureworks_sales_customer = pd.read_sql('SELECT * FROM Sales.Customer', adventureworks_conn)
adventureworks_sales_personcreditcard = pd.read_sql('SELECT * FROM Sales.PersonCreditCard', adventureworks_conn)
adventureworks_sales_salesorderdetail = pd.read_sql('SELECT * FROM Sales.SalesOrderDetail', adventureworks_conn)
adventureworks_sales_salesorderheader = pd.read_sql('SELECT * FROM Sales.SalesOrderHeader', adventureworks_conn)
adventureworks_sales_salesorderhearerrsaleseason = pd.read_sql('SELECT * FROM Sales.SalesOrderHeaderSalesReason', adventureworks_conn)
adventureworks_sales_salesperson = pd.read_sql('SELECT * FROM Sales.SalesPerson', adventureworks_conn)
adventureworks_sales_salespersonquotahistory = pd.read_sql('SELECT * FROM Sales.SalesPersonQuotaHistory', adventureworks_conn)
adventureworks_sales_salesreason = pd.read_sql('SELECT * FROM Sales.SalesReason', adventureworks_conn)
adventureworks_sales_salestaxrate = pd.read_sql('SELECT * FROM Sales.SalesTaxRate', adventureworks_conn)
adventureworks_sales_salesterritory = pd.read_sql('SELECT * FROM Sales.SalesTerritory', adventureworks_conn)
adventureworks_sales_salesterritoryhistory = pd.read_sql('SELECT * FROM Sales.SalesTerritoryHistory', adventureworks_conn)
adventureworks_sales_shoppingcartitem = pd.read_sql('SELECT * FROM Sales.ShoppingCartItem', adventureworks_conn)
adventureworks_sales_specialoffer = pd.read_sql('SELECT * FROM Sales.SpecialOffer', adventureworks_conn)
adventureworks_sales_specialofferproduct = pd.read_sql('SELECT * FROM Sales.SpecialOfferProduct', adventureworks_conn)
adventureworks_sales_store = pd.read_sql('SELECT * FROM Sales.Store', adventureworks_conn)

adventureworks_conn.close()

## Combining the data
ORDER MATTERS, CAUSE SURROGATE KEYS
SK STILL NEEDS TO BE DONE

### Departments
Combining aenc and adventureworks department data

In [102]:
# adding DEPARTMENT_source_database columns to the dataframes
aenc_department['DEPARTMENT_source_database'] = 'aenc'
adventureworks_humanresources_department['DEPARTMENT_source_database'] = 'adventureworks'

# combining all department data
departments = pd.concat([aenc_department, adventureworks_humanresources_department], ignore_index=True)

# combining name and department name columns to create a name column
departments['DEPARTMENT_DEPARTMENT_DeptName'] = departments['Name'].combine_first(departments['dept_name'])
# combining dept_id and DepartmentID columns
departments['DEPARTMENT_DEPARTMENT_DeptID'] = departments['dept_id'].combine_first(departments['DepartmentID'])

# dropping the redundant columns
drop_modified_date_rowguid(departments)
departments.drop(columns=['dept_id', 'Name', 'dept_name', 'DepartmentID'], inplace=True)

# renaming the remaining columns
departments.rename(columns={'dept_head_id': 'DEPARTMENT_DEPARTMENT_DeptHeadID', 'GroupName': 'DEPARTMENT_DEPARTMENT_GroupName'}, inplace=True)

departments.head()

,DEPARTMENT_DEPARTMENT_DeptHeadID,DEPARTMENT_source_database,DEPARTMENT_DEPARTMENT_GroupName,DEPARTMENT_DEPARTMENT_DeptName,DEPARTMENT_DEPARTMENT_DeptID
0,501.0,aenc,NaN,R & D,100.0
1,902.0,aenc,NaN,Sales,200.0
2,1293.0,aenc,NaN,Finance,300.0
3,1576.0,aenc,NaN,Marketing,400.0
4,703.0,aenc,NaN,Shipping,500.0


### BusinessEntities

In [103]:
# Combining the adventureworks Person.BusinessEntity, BusinessEntityContact and ContactType data
businessentities= pd.merge(adventureworks_person_businessentity, adventureworks_person_businessentitycontact, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('_person_businessentity', '_businessentitycontact'), how="outer")

businessentities = pd.merge(businessentities, adventureworks_person_contacttype, left_on='ContactTypeID', right_on='ContactTypeID', suffixes=('', '_contacttype'), how="outer")

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(businessentities)

# renaming the columns
businessentities.rename(columns={'BusinessEntityID': 'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID', 'PersonID': 'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID', 'ContactTypeID': 'BUSINESSENTITY_CONTACTTYPE_ContactTypeID', 'Name' : 'BUSINESSENTITY_CONTACTTYPE_Name'}, inplace=True)

businessentities.head()

,BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID,BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID,BUSINESSENTITY_CONTACTTYPE_ContactTypeID,BUSINESSENTITY_CONTACTTYPE_Name
0,NaN,NaN,1.0,Accounting Manager
1,1510.0,1509.0,2.0,Assistant Sales Agent
2,1518.0,1517.0,2.0,Assistant Sales Agent
3,1522.0,1521.0,2.0,Assistant Sales Agent
4,1528.0,1527.0,2.0,Assistant Sales Agent


### BusinessEntityAddresses

In [104]:
# Combining the adventureworks Person.BusinessEntityAddress, Address and AddressType data
businessentityaddresses = pd.merge(adventureworks_person_businessentityaddress, adventureworks_person_address, left_on='AddressID', right_on='AddressID', suffixes=('', '_address'), how="outer")

businessentityaddresses = pd.merge(businessentityaddresses, adventureworks_person_address_type, left_on='AddressTypeID', right_on='AddressTypeID', suffixes=('', '_address_type'), how="outer")


# dropping the modified date and rowguid columns
drop_modified_date_rowguid(businessentityaddresses)

# renaming the columns
businessentityaddresses.rename(columns={'BusinessEntityID': 'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID', 'AddressID': 'BUSINESSENTITYADDRESS_ADDRESS_AddressID', 'AddressTypeID': 'BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID', 'AddressLine1' : 'BUSINESSENTITYADDRESS_ADDRESS_AddressLine1', 'AddressLine2' : 'BUSINESSENTITYADDRESS_ADDRESS_AddressLine2', 'City' : 'BUSINESSENTITYADDRESS_ADDRESS_City', 'StateProvinceID' : 'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID', 'PostalCode' : 'BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE', 'SpatialLocation' : 'BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation', 'Name' : 'BUSINESSENTITYADDRESS_ADDRESSTYPE_Name'}, inplace=True)

businessentityaddresses.head()

,BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID,BUSINESSENTITYADDRESS_ADDRESS_AddressID,BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID,BUSINESSENTITYADDRESS_ADDRESS_AddressLine1,BUSINESSENTITYADDRESS_ADDRESS_AddressLine2,BUSINESSENTITYADDRESS_ADDRESS_City,BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID,BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE,BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation,BUSINESSENTITYADDRESS_ADDRESSTYPE_Name
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,Billing
1,12.0,1.0,2,1970 Napa Ct.,None,Bothell,79.0,98011,POINT (-122.164644615406 47.7869921906598),Home
2,123.0,2.0,2,9833 Mt. Dias Blv.,None,Bothell,79.0,98011,POINT (-122.250185528911 47.6867097047995),Home
3,285.0,3.0,2,7484 Roundtree Drive,None,Bothell,79.0,98011,POINT (-122.274625789912 47.7631154083121),Home
4,251.0,4.0,2,9539 Glenside Dr,None,Bothell,79.0,98011,POINT (-122.335726442416 47.7392386259644),Home


### People

In [105]:
# combining person data from adventureworks HumanResources.Person, PersonPhone, PhoneNumberType, EmailAddress and Password
people = pd.merge(adventureworks_person_person, adventureworks_person_personphone, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes= ('_person', '_personphone'), how="outer")

people = pd.merge(people, adventureworks_person_phonenumbertype, left_on='PhoneNumberTypeID', right_on='PhoneNumberTypeID', suffixes=('', '_phonenumbertype'), how="outer")

people = pd.merge(people, adventureworks_person_emailaddress, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('','_emailaddress'), how="outer")

people = pd.merge(people, adventureworks_person_password, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('','_password'), how="outer")

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(people)

# renaming the columns
people.rename(columns={'BusinessEntityID': 'PERSON_PERSON_BusinessEntityID', 'PersonType': 'PERSON_PERSON_PersonType', 'NameStyle': 'PERSON_PERSON_NameStyle', 'Title': 'PERSON_PERSON_Title', 'FirstName': 'PERSON_PERSON_FirstName', 'MiddleName' : 'PERSON_PERSON_MiddleName', 'LastName' : 'PERSON_PERSON_LastName', 'Suffix': 'PERSON_PERSON_Suffix', 'EmailPromotion' : 'PERSON_PERSON_EmailPromotion', 'AdditionalContactInfo' : 'PERSON_PERSON_AdditionalContactInfo', 'Demographics' : 'PERSON_PERSON_Demographics', 'PhoneNumber' : 'PERSON_PERSONPHONE_PhoneNumber', 'PhoneNumberTypeID' : 'PERSON_PHONENUMBERTYPE_PhoneNumberTypeID', 'Name' : 'PERSON_PHONENUMBERTYPE_Name', 'EmailAddressID': 'PERSON_EMAILADDRESS_EmailAddressID', 'EmailAddress' : 'PERSON_EMAILADDRESS_EmailAddress', 'PasswordHash' : 'PERSON_PASSWORD_PasswordHash', 'PasswordSalt' : 'PERSON_PASSWORD_PasswordSalt'}, inplace=True)

people.head()

,PERSON_PERSON_BusinessEntityID,PERSON_PERSON_PersonType,PERSON_PERSON_NameStyle,PERSON_PERSON_Title,PERSON_PERSON_FirstName,PERSON_PERSON_MiddleName,PERSON_PERSON_LastName,PERSON_PERSON_Suffix,PERSON_PERSON_EmailPromotion,PERSON_PERSON_AdditionalContactInfo,PERSON_PERSON_Demographics,PERSON_PERSONPHONE_PhoneNumber,PERSON_PHONENUMBERTYPE_PhoneNumberTypeID,PERSON_PHONENUMBERTYPE_Name,PERSON_EMAILADDRESS_EmailAddressID,PERSON_EMAILADDRESS_EmailAddress,PERSON_PASSWORD_PasswordHash,PERSON_PASSWORD_PasswordSalt
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",697-555-0142,1,Cell,1,ken0@adventure-works.com,pbFwXWE99vobT6g+vPWFy93NtUU/orrIWafF01hccfM=,bE3XiWw=
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",819-555-0175,3,Work,2,terri0@adventure-works.com,bawRVNrZQYQ05qF05Gz6VLilnviZmrqBReTTAGAudm0=,EjJaC3U=
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",212-555-0187,1,Cell,3,roberto0@adventure-works.com,8BUXrZfDqO1IyHCWOYzYmqN1IhTUn3CJMpdx/UCQ3iY=,wbPZqMw=
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",612-555-0100,1,Cell,4,rob0@adventure-works.com,SjLXpiarHSlz+6AG+H+4QpB/IPRzras/+9q/5Wr7tf8=,PwSunQU=
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",849-555-0139,1,Cell,5,gail0@adventure-works.com,8FYdAiY6gWuBsgjCFdg0UibtsqOcWHf9TyaHIP7+paA=,qYhZRiM=


### JobCandidate

In [106]:
jobcandidates = adventureworks_humanresources_jobcandidate

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(jobcandidates)

# renaming the columns
jobcandidates.rename(columns={'JobCandidateID': 'JOBCANDIDATE_JOBCANDIDATE_JobCandidateID', 'BusinessEntityID': 'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID', 'Resume': 'JOBCANDIDATE_JOBCANDIDATE_Resume'}, inplace=True)

jobcandidates.head()

,JOBCANDIDATE_JOBCANDIDATE_JobCandidateID,JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID,JOBCANDIDATE_JOBCANDIDATE_Resume
0,1,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
1,2,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
2,3,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
3,4,274.0,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."
4,5,NaN,"<ns:Resume xmlns:ns=""http://schemas.microsoft...."


### EmployeeDepartmentHistory

In [107]:
employeedepartmenthistories = adventureworks_humanresources_employeedepartmenthistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(employeedepartmenthistories)

# renaming the columns
employeedepartmenthistories.rename(columns={'BusinessEntityID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID', 'DepartmentID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID', 'ShiftID': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID', 'StartDate': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate', 'EndDate': 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate'}, inplace=True)

employeedepartmenthistories.head()

,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate,EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate
0,1,16,1,2009-01-14,None
1,2,1,1,2008-01-31,None
2,3,1,1,2007-11-11,None
3,4,1,1,2007-12-05,2010-05-30
4,4,2,1,2010-05-31,None


### EmployeePayHistory

In [108]:
employeepayhistories = adventureworks_humanresources_employeepayhistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(employeepayhistories)

# renaming the columns
employeepayhistories.rename(columns={'BusinessEntityID': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID', 'RateChangeDate': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate', 'Rate': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate', 'PayFrequency': 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency'}, inplace=True)

employeepayhistories.head()

,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate,EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency
0,1,2009-01-14,125.5000,2
1,2,2008-01-31,63.4615,2
2,3,2007-11-11,43.2692,2
3,4,2007-12-05,8.6200,2
4,4,2010-05-31,23.7200,2


### Shift

In [109]:
shifts = adventureworks_humanresources_shift

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(shifts)

# renaming the columns
shifts.rename(columns={'ShiftID': 'SHIFT_SHIFT_ShiftID', 'Name': 'SHIFT_SHIFT_Name', 'StartTime': 'SHIFT_SHIFT_StartTime', 'EndTime': 'SHIFT_SHIFT_EndTime'}, inplace=True)

shifts.head()

,SHIFT_SHIFT_ShiftID,SHIFT_SHIFT_Name,SHIFT_SHIFT_StartTime,SHIFT_SHIFT_EndTime
0,1,Day,07:00:00.0000000,15:00:00.0000000
1,2,Evening,15:00:00.0000000,23:00:00.0000000
2,3,Night,23:00:00.0000000,07:00:00.0000000


### SalesPerson

In [110]:
salespeople = adventureworks_sales_salesperson

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salespeople)

# renaming the columns
salespeople.rename(columns={'BusinessEntityID': 'SALESPERSON_SALESPERSON_BusinessEntityID', 'TerritoryID': 'SALESPERSON_SALESPERSON_TerritoryID', 'SalesQuota': 'SALESPERSON_SALESPERSON_SalesQuota', 'Bonus': 'SALESPERSON_SALESPERSON_Bonus', 'CommissionPct': 'SALESPERSON_SALESPERSON_CommissionPct', 'SalesYTD': 'SALESPERSON_SALESPERSON_SalesYTD', 'SalesLastYear': 'SALESPERSON_SALESPERSON_SalesLastYear'}, inplace=True)

salespeople.head()

,SALESPERSON_SALESPERSON_BusinessEntityID,SALESPERSON_SALESPERSON_TerritoryID,SALESPERSON_SALESPERSON_SalesQuota,SALESPERSON_SALESPERSON_Bonus,SALESPERSON_SALESPERSON_CommissionPct,SALESPERSON_SALESPERSON_SalesYTD,SALESPERSON_SALESPERSON_SalesLastYear
0,274,NaN,NaN,0.0,0.000,5.596976e+05,0.000000e+00
1,275,2.0,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06
2,276,4.0,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06
3,277,3.0,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06
4,278,6.0,250000.0,500.0,0.010,1.453719e+06,1.620277e+06


### ProductVendor

In [111]:
productvendors = adventureworks_purchasing_productvendor

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productvendors)

# renaming the columns
productvendors.rename(columns={'ProductID': 'PRODUCTVENDOR_PRODUCTVENDOR_ProductID', 'BusinessEntityID': 'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID', 'AverageLeadTime': 'PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime', 'StandardPrice': 'PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice', 'LastReceiptCost': 'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost', 'LastReceiptDate': 'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate', 'MinOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty', 'MaxOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty', 'OnOrderQty': 'PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty', 'UnitMeasureCode' : 'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode'}, inplace=True)

productvendors.head()

,PRODUCTVENDOR_PRODUCTVENDOR_ProductID,PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID,PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime,PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice,PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost,PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate,PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty,PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode
0,1,1580,17,47.87,50.2635,2011-08-29,1,5,3.0,CS
1,2,1688,19,39.92,41.9160,2011-08-29,1,5,3.0,CTN
2,4,1650,17,54.31,57.0255,2011-08-29,1,5,NaN,CTN
3,317,1578,19,28.17,29.5785,2011-08-29,100,1000,300.0,EA
4,317,1678,17,25.77,27.0585,2011-08-25,100,1000,NaN,EA


### CustomerCustomerDemo

In [112]:
customercustomerdemos = northwind_customer_customer_demo

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(customercustomerdemos)

# renaming the columns
customercustomerdemos.rename(columns={'CustomerID': 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID', 'CustomerTypeID': 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID'}, inplace=True)

customercustomerdemos.head()

,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID


### CustomerDemographics

In [113]:
customerdemographics = northwind_customer_demographics

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(customerdemographics)

# renaming the columns
customerdemographics.rename(columns={'CustomerTypeID': 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID', 'CustomerDesc': 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc'}, inplace=True)

customerdemographics.head()

,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc


### SalesTerritoryHistory

In [114]:
salesterritoryhistories = adventureworks_sales_salesterritoryhistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salesterritoryhistories)

# renaming the columns
salesterritoryhistories.rename(columns={'BusinessEntityID': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID', 'TerritoryID': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID', 'StartDate': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate', 'EndDate': 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate'}, inplace=True)

salesterritoryhistories.head()

,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate,SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate
0,275,2,2011-05-31,2012-11-29
1,275,3,2012-11-30,NaT
2,276,4,2011-05-31,NaT
3,277,3,2011-05-31,2012-11-29
4,277,2,2012-11-30,NaT


### ProductListPriceHistory

In [115]:
productlistpricehistories = adventureworks_production_productlistpricehistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productlistpricehistories)

# renaming the columns
productlistpricehistories.rename(columns={'ProductID': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID', 'StartDate': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate', 'EndDate': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate', 'ListPrice': 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice'}, inplace=True)

productlistpricehistories.head()

,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate,PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice
0,707,2011-05-31,2012-05-29,33.6442
1,707,2012-05-30,2013-05-29,33.6442
2,707,2013-05-30,NaT,34.9900
3,708,2011-05-31,2012-05-29,33.6442
4,708,2012-05-30,2013-05-29,33.6442


### ProductCostHistory

In [116]:
productcosthistories = adventureworks_production_productcosthistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(productcosthistories)

# renaming the columns
productcosthistories.rename(columns={'ProductID': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID', 'StartDate': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate', 'EndDate': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate', 'StandardCost': 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost'}, inplace=True)

productcosthistories.head()

,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate,PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost
0,707,2011-05-31,2012-05-29,12.0278
1,707,2012-05-30,2013-05-29,13.8782
2,707,2013-05-30,NaT,13.0863
3,708,2011-05-31,2012-05-29,12.0278
4,708,2012-05-30,2013-05-29,13.8782


### ShoppingCartItem

In [117]:
shoppingcartitems = adventureworks_sales_shoppingcartitem

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(shoppingcartitems)

# renaming the columns
shoppingcartitems.rename(columns={'ShoppingCartItemID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID', 'ShoppingCartID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID', 'Quantity': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity', 'ProductID': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID', 'DateCreated': 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated'}, inplace=True)

shoppingcartitems.head()

,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity,SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID,SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated
0,2,14951,3,862,2013-11-09 17:54:07.603
1,4,20621,4,881,2013-11-09 17:54:07.603
2,5,20621,7,874,2013-11-09 17:54:07.603


### SalesPersonQuotaHistory

In [118]:
salespersonquotahistories = adventureworks_sales_salespersonquotahistory

# dropping the modified date and rowguid columns
drop_modified_date_rowguid(salespersonquotahistories)

# renaming the columns
salespersonquotahistories.rename(columns={'BusinessEntityID': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID', 'QuotaDate': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate', 'SalesQuota': 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota'}, inplace=True)

salespersonquotahistories.head()

,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate,SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota
0,274,2011-05-31,28000.0
1,274,2011-08-31,7000.0
2,274,2011-12-01,91000.0
3,274,2012-02-29,140000.0
4,274,2012-05-30,70000.0


### Products (TODO CHANGE)

In [119]:
# renaming aenc product columns to match the other product data
aenc_product.rename(columns={'id': 'ProductID', 'name' : 'ProductName', 'description':'Description' , 'prod_size' : 'ProdSize', 'color': 'Color', 'quantity': 'Quantity','unit_price' : 'UnitPrice'}, inplace=True)

aenc_product.head()

,ProductID,ProductName,Description,ProdSize,Color,Quantity,UnitPrice,picture_name,Category
0,300,Tee Shirt,Tank Top,Small,White,18,9,tshirtw.bmp,Clothes
1,301,Tee Shirt,V-neck,Medium,Orange,39,14,tshirto.bmp,Clothes
2,302,Tee Shirt,Crew Neck,One size fits all,Black,72,14,tshirtb.bmp,Clothes
3,400,Baseball Cap,Cotton Cap,One size fits all,Black,92,9,capb.bmp,Accessories
4,401,Baseball Cap,Wool cap,One size fits all,White,12,10,capw.bmp,Accessories


In [120]:
# combining all adventureworks product data
adventureworks_combined_products = pd.concat([adventureworks_production_product, adventureworks_production_productcategory, adventureworks_production_productsubcategory, adventureworks_production_productdescription, adventureworks_production_productdocument, adventureworks_production_productmodel, adventureworks_production_productmodelillustration, adventureworks_production_productmodelproductdescriptionculture, adventureworks_production_productphoto, adventureworks_production_productproductphoto  ], ignore_index=True)

adventureworks_combined_products.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,CatalogDescription,Instructions,IllustrationID,CultureID,ProductPhotoID,ThumbNailPhotoHexString,ThumbNailPhotoFileName,LargePhotoHexString,LargePhotoFileName,Primary
0,1.0,Adjustable Race,AR-5381,False,False,None,1000.0,750.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Bearing Ball,BA-8327,False,False,None,1000.0,750.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,BB Ball Bearing,BE-2349,True,False,None,800.0,600.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Headset Ball Bearings,BE-2908,False,False,None,800.0,600.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,316.0,Blade,BL-2036,True,False,None,800.0,600.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# Combining all product data
products = pd.concat([northwind_products, aenc_product, adventureworks_combined_products], ignore_index=True)

# replacing the document node with documentnodestring
products['DocumentNode'] = products['DocumentNodeString']
products.drop(columns=['DocumentNodeString'], inplace=True)

# applying the data types to the columns
products['ProductID'] = products['ProductID'].astype(float) # casting to float seems counterintuitive but it is necessary to avoid an error in Pandas

# dropping documentnode column (need to fix later)
products.drop(columns=['DocumentNode'], inplace=True)


products.head()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,...,CatalogDescription,Instructions,IllustrationID,CultureID,ProductPhotoID,ThumbNailPhotoHexString,ThumbNailPhotoFileName,LargePhotoHexString,LargePhotoFileName,Primary
0,1.0,Chai,1.0,1.0,10 boxes x 20 bags,18.00,39.0,0.0,10.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Chang,1.0,1.0,24 - 12 oz bottles,19.00,17.0,40.0,25.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Aniseed Syrup,1.0,2.0,12 - 550 ml bottles,10.00,13.0,70.0,25.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Chef Anton's Cajun Seasoning,2.0,2.0,48 - 6 oz jars,22.00,53.0,0.0,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Chef Anton's Gumbo Mix,2.0,2.0,36 boxes,21.35,0.0,0.0,0.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Regions (TODO CHANGE)

In [122]:
# Combining all region data
regions = pd.concat([northwind_region, aenc_region, adventureworks_person_stateprovince], ignore_index=True)

# combining regiondescription and region columns to create a name column (one is always None)
regions['RegionName'] = regions['RegionDescription'].combine_first(regions['region'])

# dropping the other columns
regions.drop(columns=['RegionDescription', 'region'], inplace=True)


regions.head()

,RegionID,StateProvinceID,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,Name,TerritoryID,rowguid,ModifiedDate,RegionName
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Eastern ...
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Western ...
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Northern ...
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Southern ...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Canada


### Customers (TODO CHANGE)

In [123]:
# combining northwind customer data
northwind_combined_customer = pd.concat([northwind_customers, northwind_customer_customer_demo, northwind_customer_demographics], ignore_index=True)

# renaming northwind customer columns to match the other customer data
northwind_combined_customer.rename(columns={'PostalCode' : 'Zip'}, inplace=True)

northwind_combined_customer.head()

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,Zip,Country,Phone,Fax,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID,CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID,CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,NaN,NaN,NaN,NaN
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la ConstituciÃ³n 2222,MÃ©xico D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745,NaN,NaN,NaN,NaN
2,ANTON,Antonio Moreno TaquerÃ­a,Antonio Moreno,Owner,Mataderos 2312,MÃ©xico D.F.,None,05023,Mexico,(5) 555-3932,None,NaN,NaN,NaN,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750,NaN,NaN,NaN,NaN
4,BERGS,Berglunds snabbkÃ¶p,Christina Berglund,Order Administrator,BerguvsvÃ¤gen 8,LuleÃ¥,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67,NaN,NaN,NaN,NaN


In [124]:
# combining all adventureworks customer data
adventureworks_combined_customers = pd.merge(adventureworks_sales_customer, adventureworks_person_person, left_on='PersonID', right_on='BusinessEntityID', how='outer', suffixes=('_sales_customer', '_person'))

adventureworks_combined_customers = pd.merge(adventureworks_combined_customers, adventureworks_person_businessentityaddress, left_on='BusinessEntityID', right_on='BusinessEntityID', suffixes=('', '_businessentityaddress'))

adventureworks_combined_customers = pd.merge(adventureworks_combined_customers, adventureworks_person_address, left_on='AddressID', right_on='AddressID', how='left', suffixes=('', '_address'))

adventureworks_combined_customers = pd.merge(adventureworks_combined_customers, adventureworks_person_address_type, left_on='AddressTypeID', right_on='AddressTypeID', how='left', suffixes=('', '_address_type'))

adventureworks_combined_customers = pd.merge(adventureworks_combined_customers, adventureworks_person_stateprovince, left_on=['StateProvinceID', 'TerritoryID'], right_on=['StateProvinceID', 'TerritoryID'], suffixes=('', '_stateprovince'))

adventureworks_combined_customers = pd.merge(adventureworks_combined_customers, adventureworks_person_countryregion, left_on='CountryRegionCode', right_on='CountryRegionCode', suffixes=('', '_countryregion'))

# combining fist, middle and last name columns to create a contact name column
adventureworks_combined_customers['ContactName'] = adventureworks_combined_customers['FirstName'] + ' ' + adventureworks_combined_customers['MiddleName'] + ' ' + adventureworks_combined_customers['LastName']
adventureworks_combined_customers.drop(columns=['FirstName', 'MiddleName', 'LastName'], inplace=True)

# Remove any double spaces caused by missing middle names
adventureworks_combined_customers['ContactName'] = adventureworks_combined_customers['ContactName'].str.replace('  ', ' ')

# Get all columns that contain 'rowguid' in their name
columns_to_drop = adventureworks_combined_customers.filter(like='rowguid').columns

# Drop these columns
adventureworks_combined_customers.drop(columns=columns_to_drop, inplace=True)

# only keeping the most recent modified date from the two tables
adventureworks_combined_customers['ModifiedDate'] = adventureworks_combined_customers[['ModifiedDate_sales_customer', 'ModifiedDate_person', 'ModifiedDate', 'ModifiedDate_address', 'ModifiedDate_address_type', 'ModifiedDate_stateprovince', 'ModifiedDate_countryregion']].max(axis=1)

# dropping the other modified date columns
adventureworks_combined_customers.drop(columns=['ModifiedDate_sales_customer', 'ModifiedDate_person', 'ModifiedDate', 'ModifiedDate_address', 'ModifiedDate_address_type', 'ModifiedDate_stateprovince', 'ModifiedDate_countryregion'], inplace=True)

# combining PersonID and BusinessEntityID columns
adventureworks_combined_customers['PersonID'] = adventureworks_combined_customers['PersonID'].combine_first(adventureworks_combined_customers['BusinessEntityID'])
adventureworks_combined_customers.drop(columns=['BusinessEntityID'], inplace=True)

# renaming columns to match the other customer data
adventureworks_combined_customers.rename(columns={'AddressLine1' : 'Address', 'PostalCode' : 'Zip', 'Name' : 'AddressType', 'Name_stateprovince' : 'StateProvince', 'Name_countryregion': 'CountryRegion' }, inplace=True)

adventureworks_combined_customers.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,PersonType,NameStyle,Title,Suffix,EmailPromotion,...,StateProvinceID,Zip,SpatialLocation,AddressType,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,StateProvince,CountryRegion,ContactName
0,11377.0,1699.0,NaN,8.0,AW00011377,IN,False,Mr.,None,1.0,...,53,42651,POINT (7.11082410683939 51.2015555665827),Home,NW,DE,False,Nordrhein-Westfalen,Germany,David R. Robinett
1,11913.0,1700.0,NaN,9.0,AW00011913,IN,False,Ms.,None,0.0,...,77,3198,POINT (145.141451560879 -38.0612939642931),Home,VIC,AU,False,Victoria,Australia,Rebecca A. Robinson
2,11952.0,1701.0,NaN,9.0,AW00011952,IN,False,Ms.,None,2.0,...,77,3220,POINT (144.201620782255 -38.1464342680786),Home,VIC,AU,False,Victoria,Australia,Dorothy B. Robinson
3,20164.0,1702.0,NaN,10.0,AW00020164,IN,False,Ms.,None,0.0,...,14,LA1 1LN,POINT (-2.80326155845985 54.1184442932464),Home,ENG,GB,True,England,United Kingdom,Carol Ann F. Rockne
4,20211.0,1703.0,NaN,9.0,AW00020211,IN,False,Mr.,None,0.0,...,64,4169,POINT (152.9802503342 -27.4802117164592),Home,QLD,AU,False,Queensland,Australia,Scott M. Rodgers


In [125]:
# renaming aenc customer columns to match the other customer data
aenc_customer.rename(columns={'id': 'CustomerID', 'address' : 'Address', 'city':'City' , 'state' : 'State', 'zip' : 'Zip', 'phone': 'Phone', 'company_name' : 'CompanyName'}, inplace=True)

# combining fname and lname columns to create a contact name column
aenc_customer['ContactName'] = aenc_customer['fname'] + ' ' + aenc_customer['lname']
aenc_customer.drop(columns=['fname', 'lname'], inplace=True)

aenc_customer.head()

,CustomerID,Address,City,State,Zip,Phone,CompanyName,ContactName
0,101,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group,Michaels Devlin
1,102,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.,Beth Reiser
2,103,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates,Erin Niedringhaus
3,104,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.,Meghan Mason
4,105,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons,Laura McCarthy


In [126]:
# Combining all customer data
customers = pd.concat([northwind_combined_customer, aenc_customer, adventureworks_combined_customers], ignore_index=True)

customers.head()

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,Zip,Country,Phone,...,AddressTypeID,AddressLine2,StateProvinceID,SpatialLocation,AddressType,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,StateProvince,CountryRegion
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la ConstituciÃ³n 2222,MÃ©xico D.F.,None,05021,Mexico,(5) 555-4729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANTON,Antonio Moreno TaquerÃ­a,Antonio Moreno,Owner,Mataderos 2312,MÃ©xico D.F.,None,05023,Mexico,(5) 555-3932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BERGS,Berglunds snabbkÃ¶p,Christina Berglund,Order Administrator,BerguvsvÃ¤gen 8,LuleÃ¥,None,S-958 22,Sweden,0921-12 34 65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Date

In [127]:
def calculate_periods(periods_in_years): 
    return 365 * periods_in_years + (periods_in_years // 4) # divide by 4 for leap years

periods = calculate_periods(50)

date_table = {
    'DATE_ID': pd.date_range(start='1996-01-01', periods=periods).strftime('%Y%m%d').astype(int),
    'DATE_Date': pd.date_range(start='1996-01-01', periods=periods),
    'DATE_Weekday': pd.date_range(start='1996-01-01', periods=periods).strftime('%A'),
    'DATE_WeekdayNum': pd.date_range(start='1996-01-01', periods=periods).weekday + 1,
    'DATE_DayMonth': pd.date_range(start='1996-01-01', periods=periods).day,
    'DATE_DayOfYear': pd.date_range(start='1996-01-01', periods=periods).dayofyear,
    'DATE_WeekOfYear': pd.date_range(start='1996-01-01', periods=periods).isocalendar().week,
    'DATE_MonthNum': pd.date_range(start='1996-01-01', periods=periods).month,
    'DATE_MonthName': pd.date_range(start='1996-01-01', periods=periods).strftime('%B'),
    'DATE_MonthNameShort': pd.date_range(start='1996-01-01', periods=periods).strftime('%b'),
    'DATE_Quarter': pd.date_range(start='1996-01-01', periods=periods).quarter,
    'DATE_Year': pd.date_range(start='1996-01-01', periods=periods).year,
    'DATE_FirstDayOfMonth': pd.date_range(start='1996-01-01', periods=periods).to_period('M').start_time,
    'DATE_LastDayOfMonth': pd.date_range(start='1996-01-01', periods=periods).to_period('M').end_time.date,
    'DATE_YYYYMM': pd.date_range(start='1996-01-01', periods=periods).strftime('%Y-%m'),
    'DATE_WeekendIndr': pd.date_range(start='1996-01-01', periods=periods).weekday // 5
}

date_table = pd.DataFrame(date_table)
date_table['DATE_WeekendIndr'] = date_table['DATE_WeekendIndr'].replace({0: 'weekday', 1: 'weekend'})

date_table.tail()

,DATE_ID,DATE_Date,DATE_Weekday,DATE_WeekdayNum,DATE_DayMonth,DATE_DayOfYear,DATE_WeekOfYear,DATE_MonthNum,DATE_MonthName,DATE_MonthNameShort,DATE_Quarter,DATE_Year,DATE_FirstDayOfMonth,DATE_LastDayOfMonth,DATE_YYYYMM,DATE_WeekendIndr
2045-12-26,20451226,2045-12-26,Tuesday,2,26,360,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-27,20451227,2045-12-27,Wednesday,3,27,361,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-28,20451228,2045-12-28,Thursday,4,28,362,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-29,20451229,2045-12-29,Friday,5,29,363,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekday
2045-12-30,20451230,2045-12-30,Saturday,6,30,364,52,12,December,Dec,4,2045,2045-12-01,2045-12-31,2045-12,weekend


### Time

In [128]:
# Define the number of minutes in a day
minutes_in_a_day = 24 * 60

# Create the data dictionary for time dimension
time_table = {
    'TIME_ID': list(range(minutes_in_a_day)),
    'TIME_Hour': [i // 60 for i in range(minutes_in_a_day)],
    'TIME_Minute': [i % 60 for i in range(minutes_in_a_day)],
    'TIME_HourMinute': [f'{hour:02d}:{minute:02d}' for hour in range(24) for minute in range(60)],
}

# Convert the dictionary to a DataFrame
time_table = pd.DataFrame(time_table)

# # Fetch existing data from the time dimension table
# existing_data_query = "SELECT TIME_ID FROM Time"
# existing_data = pd.read_sql(existing_data_query, united_outdoors_conn)

# # Filter new data to include only records that are not already in the table
# new_data_to_insert = time_table[~time_table['TIME_ID'].isin(existing_data['TIME_ID'])]

time_table

,TIME_ID,TIME_Hour,TIME_Minute,TIME_HourMinute
0,0,0,0,00:00
1,1,0,1,00:01
2,2,0,2,00:02
3,3,0,3,00:03
4,4,0,4,00:04
...,...,...,...,...
1435,1435,23,55,23:55
1436,1436,23,56,23:56
1437,1437,23,57,23:57
1438,1438,23,58,23:58


## Loading the data into the UnitedOutdoors datawarehouse

### Departments

In [129]:
departments_dtypes = {
    'DEPARTMENT_DEPARTMENT_DeptID': Integer,
    'DEPARTMENT_DEPARTMENT_DeptName': String(100),
    'DEPARTMENT_DEPARTMENT_GroupName': String(100),
    'DEPARTMENT_DEPARTMENT_DeptHeadID': Integer,
    'DEPARTMENT_source_database': String(100)
}

# TODO dept_head_id needs to refer to an employee
departments_nk_sk_dict  = prepare_and_insert_return_sk(departments, departments_dtypes, 'Department',united_outdoors_engine, 'DEPARTMENT_DEPARTMENT_DeptID', { 'DEPARTMENT_DEPARTMENT_DeptHeadID' : {}})

Replacing natural keys with surrogate keys for column: DEPARTMENT_DEPARTMENT_DeptHeadID
Inserting data into table: Department with chunk size: 400


### BusinessEntities

In [130]:
businessentities_dtypes = {
    'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID': Integer,
    'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID': Integer,
    'BUSINESSENTITY_CONTACTTYPE_ContactTypeID': Integer,
    'BUSINESSENTITY_CONTACTTYPE_Name': String(100)
}

businessentities_nk_sk_dict = prepare_and_insert_return_sk(businessentities, businessentities_dtypes, 'BusinessEntity',united_outdoors_engine, 'BUSINESSENTITY_BUSINESSENTITY_BusinessEntityID')

Inserting data into table: BusinessEntity with chunk size: 500


### People

In [131]:
people_dtypes = {
    'PERSON_PERSON_BusinessEntityID': Integer,
    'PERSON_PERSON_PersonType': String(2),
    'PERSON_PERSON_NameStyle': BIT,
    'PERSON_PERSON_Title': String(100),
    'PERSON_PERSON_FirstName': String(100),
    'PERSON_PERSON_MiddleName': String(100),
    'PERSON_PERSON_LastName': String(100),
    'PERSON_PERSON_Suffix': String(100),
    'PERSON_PERSON_EmailPromotion': Integer,
    'PERSON_PERSON_AdditionalContactInfo': XML,
    'PERSON_PERSON_Demographics': XML,
    'PERSON_PERSONPHONE_PhoneNumber': String(100),
    'PERSON_PHONENUMBERTYPE_PhoneNumberTypeID': Integer,
    'PERSON_PHONENUMBERTYPE_Name': String(100),
    'PERSON_EMAILADDRESS_EmailAddressID': Integer,
    'PERSON_EMAILADDRESS_EmailAddress': String(100),
    'PERSON_PASSWORD_PasswordHash': LargeBinary,
    'PERSON_PASSWORD_PasswordSalt': LargeBinary
}

# Convert the 'PERSON_PASSWORD_PasswordHash' and 'PERSON_PASSWORD_PasswordSalt' columns to bytes
people['PERSON_PASSWORD_PasswordHash'] = people['PERSON_PASSWORD_PasswordHash'].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)
people['PERSON_PASSWORD_PasswordSalt'] = people['PERSON_PASSWORD_PasswordSalt'].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)

prepare_and_insert(people, people_dtypes, 'Person',united_outdoors_engine, { 'PERSON_PERSON_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: PERSON_PERSON_BusinessEntityID
Inserting data into table: Person with chunk size: 111


### Updating the BusinessEntity table
replacing the natural keys with the surrogate keys for the PersonID column

In [132]:
prepare_and_update('BusinessEntity', united_outdoors_engine, { 'BUSINESSENTITY_BUSINESSENTITYCONTACT_PersonID' : businessentities_nk_sk_dict})

Updating data in table: BusinessEntity


### BusinessEntityAddresses

In [133]:
businessentityaddresses_dtypes = {
    'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESSTYPE_AddressTypeID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESSTYPE_Name': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_AddressID': Integer,
    'BUSINESSENTITYADDRESS_ADDRESS_AddressLine1': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_AddressLine2': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_City': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_POSTALCODE': String(100),
    'BUSINESSENTITYADDRESS_ADDRESS_SpatialLocation': VARCHAR,
    'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID': Integer
}

# TODO the StateProvinceID needs to refer to sk of Territory
prepare_and_insert(businessentityaddresses, businessentityaddresses_dtypes, 'BusinessEntityAddress',united_outdoors_engine, { 'BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID' : businessentities_nk_sk_dict, 'BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID' : {}})

Replacing natural keys with surrogate keys for column: BUSINESSENTITYADDRESS_BUSINESSENTITYADDRESS_BusinessEntityID
Replacing natural keys with surrogate keys for column: BUSINESSENTITYADDRESS_ADDRESS_StateProvinceID
Inserting data into table: BusinessEntityAddress with chunk size: 200


### JobCandidate

In [134]:
jobcandidates_dtypes = {
    'JOBCANDIDATE_JOBCANDIDATE_JobCandidateID': Integer,
    'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID': Integer,
    'JOBCANDIDATE_JOBCANDIDATE_Resume': XML
}

prepare_and_insert(jobcandidates, jobcandidates_dtypes, 'JobCandidate',united_outdoors_engine, { 'JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: JOBCANDIDATE_JOBCANDIDATE_BusinessEntityID
Inserting data into table: JobCandidate with chunk size: 666


### EmployeeDepartmentHistory

In [135]:
employeedepartmenthistories_dtypes = {
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID': Integer,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_StartDate': DATE,
    'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_EndDate': DATE
}

prepare_and_insert(employeedepartmenthistories, employeedepartmenthistories_dtypes, 'EmployeeDepartmentHistory',united_outdoors_engine, { 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID' : businessentities_nk_sk_dict, 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID' : departments_nk_sk_dict, 'EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID' : {}})

Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_BusinessEntityID
Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_DepartmentID
Replacing natural keys with surrogate keys for column: EMPLOYEEDEPARTMENTHISTORY_EMPLOYEEDEPARTMENTHISTORY_ShiftID
Inserting data into table: EmployeeDepartmentHistory with chunk size: 400


### EmployeePayHistory

In [136]:
employeepayhistories_dtypes = {
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID': Integer,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_RateChangeDate': DATE,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_Rate': MONEY,
    'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_PayFrequency': Integer
}

prepare_and_insert(employeepayhistories, employeepayhistories_dtypes, 'EmployeePayHistory',united_outdoors_engine, { 'EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: EMPLOYEEPAYHISTORY_EMPLOYEEPAYHISTORY_BusinessEntityID
Inserting data into table: EmployeePayHistory with chunk size: 500


### Shift

In [137]:
shifts_dtypes = {
    'SHIFT_SHIFT_ShiftID': Integer,
    'SHIFT_SHIFT_Name': String(100),
    'SHIFT_SHIFT_StartTime': TIME,
    'SHIFT_SHIFT_EndTime': TIME
}

prepare_and_insert(shifts, shifts_dtypes, 'Shift',united_outdoors_engine)

Inserting data into table: Shift with chunk size: 500


### SalesPerson

In [138]:
salespeople_dtypes = {
    'SALESPERSON_SALESPERSON_BusinessEntityID': Integer,
    'SALESPERSON_SALESPERSON_TerritoryID': Integer,
    'SALESPERSON_SALESPERSON_SalesQuota': MONEY,
    'SALESPERSON_SALESPERSON_Bonus': MONEY,
    'SALESPERSON_SALESPERSON_CommissionPct': DECIMAL(8,4),
    'SALESPERSON_SALESPERSON_SalesYTD': MONEY,
    'SALESPERSON_SALESPERSON_SalesLastYear': MONEY
}

prepare_and_insert(salespeople, salespeople_dtypes, 'SalesPerson',united_outdoors_engine, { 'SALESPERSON_SALESPERSON_BusinessEntityID' : businessentities_nk_sk_dict, 'SALESPERSON_SALESPERSON_TerritoryID' : {}})

Replacing natural keys with surrogate keys for column: SALESPERSON_SALESPERSON_BusinessEntityID
Replacing natural keys with surrogate keys for column: SALESPERSON_SALESPERSON_TerritoryID
Inserting data into table: SalesPerson with chunk size: 285


### ProductVendor

In [139]:
productvendors_dtypes = {
    'PRODUCTVENDOR_PRODUCTVENDOR_ProductID': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_AverageLeadTime': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_StandardPrice': MONEY,
    'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptCost': MONEY,
    'PRODUCTVENDOR_PRODUCTVENDOR_LastReceiptDate': DATE,
    'PRODUCTVENDOR_PRODUCTVENDOR_MinOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_MaxOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_OnOrderQty': Integer,
    'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode': CHAR(3)
}

prepare_and_insert(productvendors, productvendors_dtypes, 'ProductVendor', united_outdoors_engine, { 'PRODUCTVENDOR_PRODUCTVENDOR_ProductID' : {}, 'PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID' : businessentities_nk_sk_dict, 'PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode' : {}})

Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_ProductID
Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_BusinessEntityID
Replacing natural keys with surrogate keys for column: PRODUCTVENDOR_PRODUCTVENDOR_UnitMeasureCode
Inserting data into table: ProductVendor with chunk size: 200


### CustomerDemographics

In [140]:
customerdemographics_dtypes = {
    'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID': NVARCHAR(10),
    'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerDesc': String(100)
}

customerdemographics_nk_sk_dict = prepare_and_insert_return_sk(customerdemographics, customerdemographics_dtypes, 'CustomerDemographic',united_outdoors_engine, 'CUSTOMERDEMOGRAPHIC_CUSTOMERDEMOGRAPHICS_CustomerTypeID')

Inserting data into table: CustomerDemographic with chunk size: 1000


### CustomerCustomerDemo

In [141]:
customercustomerdemos_dtypes = {
    'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID': NVARCHAR(10),
    'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID': NVARCHAR(10)
}

# TODO how to handle fk? the nk is nvarchar, not an integer

prepare_and_insert(customercustomerdemos, customercustomerdemos_dtypes, 'CustomerCustomerDemo',united_outdoors_engine, { 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID' : customerdemographics_nk_sk_dict, 'CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID' : {}})

Replacing natural keys with surrogate keys for column: CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerTypeID
Replacing natural keys with surrogate keys for column: CUSTOMERCUSTOMERDEMO_CUSTOMERCUSTOMERDEMO_CustomerID
Inserting data into table: CustomerCustomerDemo with chunk size: 1000


### SalesTerritoryHistory

In [142]:
salesterritoryhistories_dtypes = {
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID': Integer,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID': Integer,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_StartDate': DATE,
    'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_EndDate': DATE
}

prepare_and_insert(salesterritoryhistories, salesterritoryhistories_dtypes, 'SalesTerritoryHistory',united_outdoors_engine, { 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID' : businessentities_nk_sk_dict, 'SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID' : {}})

Replacing natural keys with surrogate keys for column: SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_BusinessEntityID
Replacing natural keys with surrogate keys for column: SALESTERRITORYHISTORY_SALESTERRITORYHISTORY_TerritoryID
Inserting data into table: SalesTerritoryHistory with chunk size: 500


### ProductListPriceHistory

In [143]:
productlistpricehistories_dtypes = {
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID': Integer,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_StartDate': DATE,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_EndDate': DATE,
    'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ListPrice': MONEY
}

prepare_and_insert(productlistpricehistories, productlistpricehistories_dtypes, 'ProductListPriceHistory',united_outdoors_engine, { 'PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID' : {}})

Replacing natural keys with surrogate keys for column: PRODUCTLISTPRICEHISTORY_PRODUCTLISTPRICEHISTORY_ProductID
Inserting data into table: ProductListPriceHistory with chunk size: 500


### ProductCostHistory

In [144]:
productcosthistories_dtypes = {
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID': Integer,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StartDate': DATE,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_EndDate': DATE,
    'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_StandardCost': MONEY
}

prepare_and_insert(productcosthistories, productcosthistories_dtypes, 'ProductCostHistory', united_outdoors_engine, { 'PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID' : {}})

Replacing natural keys with surrogate keys for column: PRODUCTCOSTHISTORY_PRODUCTCOSTHISTORY_ProductID
Inserting data into table: ProductCostHistory with chunk size: 500


### ShoppingCartItem

In [145]:
shoppingcartitems_dtypes = {
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartItemID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_Quantity': Integer,
    'SHOPPINGCARTITEM_SHOPPINGCARTITEM_DateCreated': DATE
}

prepare_and_insert(shoppingcartitems, shoppingcartitems_dtypes, 'ShoppingCartItem', united_outdoors_engine, { 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID' : {}, 'SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID' : {}})

Replacing natural keys with surrogate keys for column: SHOPPINGCARTITEM_SHOPPINGCARTITEM_ProductID
Replacing natural keys with surrogate keys for column: SHOPPINGCARTITEM_SHOPPINGCARTITEM_ShoppingCartID
Inserting data into table: ShoppingCartItem with chunk size: 400


### SalesPersonQuotaHistory

In [146]:
salespersonquotahistories_dtypes = {
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID': Integer,
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_QuotaDate': DATE,
    'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_SalesQuota': MONEY
}

prepare_and_insert(salespersonquotahistories, salespersonquotahistories_dtypes, 'SalesPersonQuotaHistory', united_outdoors_engine, { 'SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID' : businessentities_nk_sk_dict})

Replacing natural keys with surrogate keys for column: SALESPERSONQUOTAHISTORY_SALESPERSONQUOTAHISTORY_BusinessEntityID
Inserting data into table: SalesPersonQuotaHistory with chunk size: 666


### Products (TODO CHANGE)

In [147]:
products_dtypes = {
    'ProductID': Integer,
    'ProductName': NVARCHAR(50),
    'SupplierID': Integer,
    'CategoryID': Integer,
    'QuantityPerUnit': NVARCHAR(30),
    'UnitPrice': MONEY,
    'UnitsInStock': Integer,
    'UnitsOnOrder': Integer,
    'ReorderLevel': Integer,
    'Discontinued': BIT,
    'Description': NVARCHAR,
    'ProdSize': NVARCHAR(50),
    'Color': NVARCHAR(15),
    'Quantity': Integer,
    'picture_name': NVARCHAR(50),
    'Category': NVARCHAR(20),
    'Name': NVARCHAR(50),
    'ProductNumber': NVARCHAR(25),
    'MakeFlag': BIT,
    'FinishedGoodsFlag': BIT,
    'SafetyStockLevel': Integer,
    'ReorderPoint': Integer,
    'StandardCost': DECIMAL(8,4),
    'ListPrice': MONEY,
    'Size': NVARCHAR(5),
    'SizeUnitMeasureCode': CHAR(2),
    'WeightUnitMeasureCode': NVARCHAR(3),
    'Weight': DECIMAL(8,2),
    'DaysToManufacture': Integer,
    'ProductLine': CHAR(1),
    'Class': CHAR(1),
    'Style': CHAR(1),
    'ProductSubcategoryID': Integer,
    'ProductModelID': Integer,
    'SellStartDate': DATE,
    'SellEndDate': DATE,
    'DiscontinuedDate': DATE,
    'ModifiedDate': DATE,
    'ProductCategoryID': Integer,
    'ProductDescriptionID': Integer,
    'CatalogDescription': XML,
    'Instructions': XML,
    'IllustrationID': Integer,
    'CultureID': NVARCHAR(10),
    'ProductPhotoID': Integer,
    'ThumbNailPhotoHexString': String,
    'ThumbnailPhotoFileName': NVARCHAR(50),
    'LargePhotoHexString': String,
    'LargePhotoFileName': NVARCHAR(50),
    'Primary': BIT
}

prepare_and_insert(products, products_dtypes, 'Product', united_outdoors_engine)

Inserting data into table: Product with chunk size: 39


### Regions (TODO CHANGE)

In [148]:
regions_dtypes = {
    'RegionID': Integer,
    'RegionName': VARCHAR(10),
    'StateProvinceID': Integer,
    'StateProvinceCode': VARCHAR(10),
    'CountryRegionCode': CHAR(2),
    'IsOnlyStateProvinceFlag': BIT,
    'Name': VARCHAR(50),
    'TerritoryID': Integer,
    'ModifiedDate': DATE
}

regions['StateProvinceCode'] = regions['StateProvinceCode'].astype(str)

prepare_and_insert(regions, regions_dtypes, 'Region', united_outdoors_engine)

Inserting data into table: Region with chunk size: 200


### Customers (TODO CHANGE)

In [149]:
customers_dtypes = {
    'CustomerID': NVARCHAR(10),
    'CompanyName': NVARCHAR(40),
    'ContactName': NVARCHAR(60),
    'ContactTitle': NVARCHAR(30),
    'Address': NVARCHAR(60),
    'City': NVARCHAR(30),
    'Region': VARCHAR(15),
    'Zip': VARCHAR(10),
    'Country': VARCHAR(15),
    'Phone': VARCHAR(24),
    'Fax': VARCHAR(24),
    'CustomerTypeID': VARCHAR,
    'CustomerDesc': VARCHAR(100),
    'State': CHAR(2),
    'PersonID': Integer,
    'StoreID': Integer,
    'TerritoryID': Integer,
    'AccountNumber': CHAR(10),
    'PersonType': VARCHAR(2),
    'NameStyle': BIT,
    'Title': VARCHAR(10),
    'Suffix': VARCHAR(10),
    'EmailPromotion': Integer,
    'AdditionalContactInfo': XML,
    'Demographics': XML,
    'AddressID': Integer,
    'AddressTypeID': Integer,
    'AddressLine2': VARCHAR(60),
    'StateProvinceID': Integer,
    'SpatialLocation': VARCHAR,  # SQLAlchemy does not support the GEOGRAPHY data type
    'AddressType': VARCHAR(50),
    'StateProvinceCode': VARCHAR(10),
    'CountryRegionCode': CHAR(2),
    'IsOnlyStateProvinceFlag': BIT,
    'StateProvince': VARCHAR(50),
    'CountryRegion': VARCHAR(50)
}

# applying the data types to the columns
customers['CustomerID'] = customers['CustomerID'].astype(str)
# causes error with customerdemo for some reason, not worth fixing atm since customers needs to be fixed
#prepare_and_insert(customers, customers_dtypes, 'Customer', united_outdoors_engine) 

## Date

In [151]:
date_dtypes = {
    'DATE_ID': Integer,
    'DATE_Date': DATE,
    'DATE_Weekday': VARCHAR(10),
    'DATE_WeekdayNum': Integer,
    'DATE_DayMonth': Integer,
    'DATE_DayOfYear': Integer,
    'DATE_WeekOfYear': Integer,
    'DATE_MonthNum': Integer,
    'DATE_MonthName': VARCHAR(10),
    'DATE_MonthNameShort': CHAR(10),
    'DATE_Quarter': Integer,
    'DATE_Year': Integer,
    'DATE_FirstDayOfMonth': DATE,
    'DATE_LastDayOfMonth': DATE,
    'DATE_YYYYMM': CHAR(10),
    'DATE_WeekendIndr': CHAR(15)
}

prepare_and_insert(date_table, date_dtypes, 'Date', united_outdoors_engine)

Inserting data into table: Date with chunk size: 125


## Time

In [152]:
time_dtypes = {
    'TIME_ID': Integer,
    'TIME_Hour': Integer,
    'TIME_Minute': Integer,
    'TIME_HourMinute': VARCHAR(10),
}

prepare_and_insert(time_table, time_dtypes, 'Time', united_outdoors_engine)

Inserting data into table: Time with chunk size: 500


## Constraints
altering the tables to add the (foreign key) constraints

In [153]:
# opening the UnitedOutdoors_constraints.sql file
with open('sql/UnitedOutdoors_constraints.sql', 'r') as file:
    sql_script = file.read()

# Execute the script
split_and_execute_sql_script(sql_script, united_outdoors_engine)

## Closing connections

In [154]:
try:
    united_outdoors_engine.dispose()
    northwind_engine.dispose()
    aenc_engine.dispose()
    adventureworks_engine.dispose()
except OperationalError as e:
        print(f'Error: {e}')

# Time elapsed

In [155]:
end_time = time.time()

print(f'Time elapsed: {end_time - start_time} seconds')

Time elapsed: 428.24245953559875 seconds
